In [ ]:
!apt-get install ffmpeg
!pip install pydub


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install transformers sentencepiece speechrecognition pydub openai


In [ ]:
import os
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BartTokenizer, BartForConditionalGeneration



In [ ]:
from pydub import AudioSegment
import speech_recognition as sr
from google.colab import files

def convert_to_wav(filename):
    audio = AudioSegment.from_file(filename)
    wav_filename = filename.split('.')[0] + ".wav"
    audio.export(wav_filename, format="wav")
    return wav_filename

def transcribe_audio_file():
    uploaded = files.upload()  # User uploads an audio file
    filename = list(uploaded.keys())[0]

    # Convert to WAV if necessary
    if not filename.endswith(".wav"):
        filename = convert_to_wav(filename)

    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
    transcript = recognizer.recognize_google(audio)
    return transcript

transcript = transcribe_audio_file()
print("📌 **Transcribed Meeting:**\n", transcript)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-b488f52f4760>", line 25, in <cell line: 0>
    transcript = transcribe_audio_file()
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-4-b488f52f4760>", line 12, in transcribe_audio_file
    uploaded = files.upload()  # User uploads an audio file
               ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 72, in upload
    uploaded_files = _upload_files(multiple=True)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 164, in _upload_files
    result = _output.eval_js(
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/output/_js.py", line 40, in eval_js
    return _message.read_reply_from_input(r

TypeError: object of type 'NoneType' has no len()

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "philschmid/bart-large-cnn-samsum"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


In [ ]:
def summarize_meeting(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs.input_ids, max_length=200, min_length=50, length_penalty=2.0, num_beams=4)
    raw_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Add structure
    return format_summary(raw_summary)

# Function: Format Summary in a Practical Way
def format_summary(text):
    return f"""
### 📝 **Meeting Summary**
**📌 Key Topics Discussed:**
{text}

**✅ Decisions Made:**
- [Auto-detect decisions manually or improve NLP for it]

**🚀 Action Items & Next Steps:**
- {extract_action_items(text)}
"""

# Function: Extract Action Items
def extract_action_items(text):
    action_phrases = re.findall(r"(\b[A-Z][a-z]+ \b(?:will|needs to|should|must|is responsible for) .+?)\.", text)

    if not action_phrases:
        return "No clear action items detected."

    return "\n- ".join(action_phrases)

In [ ]:
def extract_action_items(text):
    prompt = f"Extract the action items from this meeting transcript:\n{text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs.input_ids, max_length=100, num_beams=4)
    # Return both the decoded action items and summary_ids
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True), summary_ids

# Call the function and unpack the returned values
action_items, summary_ids = extract_action_items(transcript)
print("\n🔹 **Practical Summary:**\n", summary_ids)
print("📌 **AI-Extracted Action Items:**\n", action_items)

In [ ]:
!pip install gradio



In [ ]:
!pip install --upgrade gradio


In [ ]:

def process_audio(audio_path):
    transcription = transcribe_audio_file(audio_path)
    summary = summarize_meeting(transcription)
    return transcription, summary

# Gradio UI
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(label="Transcription"), gr.Textbox(label="Summary")],
    title="Audio Meeting Summarizer",
    description="Upload an MP3 file to get a transcription and summarized text."
)

# Run Gradio UI
iface.launch(share=True, debug=True)
